# 羽毛球3D轨迹可视化（示例与说明）
本Notebook演示如何加载数据、绘制标准3D羽毛球场地、可视化多条轨迹，并播放动画。

In [ ]:
!pip -q install plotly pandas numpy 'nbformat>=4.2.0'

## 1. 导入与依赖

In [1]:
from pathlib import Path
import json
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import sys
# sys.path.append(str(Path('.').resolve()))
# sys.path.append('./')
from badminton_vis import (
    CourtSpec, load_from_json_file, load_from_json_obj, load_from_dataframe,
    sort_and_clean, smooth, interpolate_time, make_figure, make_animation,
    save_html, make_parabola_example
)

## 2. 数据格式与示例
示例JSON位于 `/mnt/data/badminton_3d_vis/sample_trajectory.json`，符合规格中的字段定义。

In [2]:
sample_path = './sample_trajectory.json'
with open(sample_path, 'r', encoding='utf-8') as f:
    sample_obj = json.load(f)
sample_obj


{'msgtype': 'pred_track',
 'pid': 'sample1',
 'positions': [{'frame_id': 0,
   'ts': 0.0,
   'pos': {'x': -3.0, 'y': 0.0, 'z': 0.05}},
  {'frame_id': 1,
   'ts': 10.0,
   'pos': {'x': -2.9240506329113924,
    'y': 0.06356013357737149,
    'z': 0.24995994231693686}},
  {'frame_id': 2,
   'ts': 20.0,
   'pos': {'x': -2.848101265822785,
    'y': 0.12671841952577198,
    'z': 0.4935106553436952}},
  {'frame_id': 3,
   'ts': 30.0,
   'pos': {'x': -2.7721518987341773,
    'y': 0.1890755508263239,
    'z': 0.730652139080275}},
  {'frame_id': 4,
   'ts': 40.0,
   'pos': {'x': -2.6962025316455698,
    'y': 0.2502372856177817,
    'z': 0.961384393526678}},
  {'frame_id': 5,
   'ts': 50.0,
   'pos': {'x': -2.620253164556962,
    'y': 0.30981693972051283,
    'z': 1.1857074186829037}},
  {'frame_id': 6,
   'ts': 60.0,
   'pos': {'x': -2.5443037974683547,
    'y': 0.3674378313783824,
    'z': 1.4036212145489508}},
  {'frame_id': 7,
   'ts': 70.0,
   'pos': {'x': -2.4683544303797467,
    'y': 0.4227

## 3. 加载与解析
支持从JSON对象/文件或DataFrame加载，自动生成 `pid, frame_id, ts, x, y, z, vx, vy, vz` 字段。

In [3]:
trajs = load_from_json_file(sample_path)
df = pd.concat([t.to_dataframe() for t in trajs], ignore_index=True)
df = sort_and_clean(df)
df.head()


KeyboardInterrupt: 

## 4. 绘制静态3D场地与轨迹

In [ ]:
court = CourtSpec()  # 可传入自定义尺寸
fig = make_figure(df, court=court, title='Badminton 3D Trajectories - Static')
fig.show()


## 5. 轨迹动画（根据时间戳ts）

In [ ]:
anim_fig = make_animation(df, court=CourtSpec(), title='Badminton 3D Trajectory Animation', time_col='ts')
anim_fig.show()


## 6. 多轨迹演示与平滑/插值
这里创建另一条仿真轨迹并与示例轨迹一起显示，同时展示平滑与按固定时间步长插值。

In [ ]:
# 生成另一条抛物线轨迹
demo_trajs = make_parabola_example(pid='demo2', apex=(0.0, -1.0, 6.0), span_x=7.0)
df2 = pd.concat([t.to_dataframe() for t in demo_trajs], ignore_index=True)
df_multi = pd.concat([df, df2], ignore_index=True)
df_multi = sort_and_clean(df_multi)

# 可选：平滑
df_smooth = smooth(df_multi, window=5)

# 可选：按固定时间步插值（需要ts列）
df_interp = interpolate_time(df_smooth, freq_ms=20)

fig_multi = make_figure(df_interp, court=CourtSpec(), title='Multiple Trajectories (Smoothed & Interpolated)')
fig_multi.show()


## 7. 导出（HTML）

In [ ]:
export_path = '/mnt/data/badminton_3d_vis/trajectory_visualization.html'
save_html(fig_multi, export_path)
export_path


## 8. 常见问题（FAQ）
- **坐标单位**：默认以米为单位。
- **坐标系**：原点在场地中心地面，x为场地长度方向，y为宽度方向，z向上。
- **自定义场地**：通过 `CourtSpec(length=..., width_doubles=..., net_height_center=...)` 调整。
- **数据来源**：可使用 `load_from_json_file/json_obj/dataframe` 加载；DataFrame方式可映射自定义列名。
- **动画时间**：优先使用 `ts`（毫秒）；如缺失，将按采样序号合成时间。
- **导出**：使用 `save_html(fig, path)` 导出可交互HTML，便于分享。